In [1]:
# Install necessary libraries
!pip install transformers datasets unsloth

  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 82.2 MB/s eta 0:00:00
Using cached wheel-0.45.1-py3-none-any.whl (72 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 103.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 104.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 101.4 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.37.1
    Uninstalling wheel-0.37.1:
      Successfully uninstalled wheel-0.37.1
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.3
    Uninstalling protobuf-5.29.3:
      Successfully uninstalled protobuf-5.29.3


In [4]:
# Load the LLaMA 3B model and tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = 'unsloth/Llama-3.2-3B'
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
from unsloth import FastLanguageModel

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

/tmp/ipykernel_464718/2882989549.py:1: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.9: Fast Llama patching. Transformers: 4.50.0.dev0.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.64 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.3.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
# import json
# from datasets import Dataset
# import pandas as pd

# # Load the JSON file
# with open('./wikiTQ_data/wikiTQ_all_splits.json', 'r') as f:
#     data = json.load(f)

# # Extract the relevant fields
# formatted_data = []
# data = data['train']
# for item in data:
#     formatted_data.append({
#         'text': f"Please read the following table and answer the questions. question: {item['question']}\n{item['tables']['bracket']}",
#         'output': item['answer']
#     })

# # Create a Hugging Face dataset
# print(len(formatted_data))
# dataset = Dataset.from_pandas(pd.DataFrame(formatted_data))

# # Save the dataset to disk
# dataset.save_to_disk('formatted_dataset')

11321


Saving the dataset (1/1 shards): 100%|██████████| 11321/11321 [00:00<00:00, 459072.60 examples/s]


In [8]:
import json
from datasets import Dataset
import pandas as pd

# Load the JSON file
with open('./tabFact_data/tabFact.json', 'r') as f:
    data = json.load(f)

# Extract the relevant fields
formatted_data = []
data = data['train']
data = data[:len(data)//4]
for item in data:
    formatted_data.append({
        'input': f"Please read the following table and answer the questions. question: {item['question']}\n{item['tables']['bracket']}",
        'output': item['answer']
    })
# Create a Hugging Face dataset
print(len(formatted_data))
dataset = Dataset.from_pandas(pd.DataFrame(formatted_data))
# Save the dataset to disk
dataset.save_to_disk('formatted_dataset')

23070


Saving the dataset (1/1 shards): 100%|██████████| 23070/23070 [00:00<00:00, 527722.08 examples/s]


In [22]:
import json
import pandas as pd

# Load the JSON file
with open('./tabFact_data/tabFact.json', 'r') as f:
    data = json.load(f)

# Extract relevant fields
formatted_data = []
data = data['train']  # Access the training split
data = data[:len(data)//4]
# Limit to 1/4 of the data for testing

for item in data:
    formatted_data.append({
        'prompt': f"{item['question']}\n{item['tables']['bracket']}",
        'completion': str(item['answer'])
    })

# Save as JSONL
jsonl_path = "formatted_dataset.jsonl"

with open(jsonl_path, "w", encoding="utf-8") as f:
    for entry in formatted_data:
        f.write(json.dumps(entry) + "\n")

print(f"JSONL file saved as {jsonl_path}")

JSONL file saved as formatted_dataset.jsonl


In [9]:
import json
import pandas as pd

# Load the JSON file
with open('./tabFact_data/tabFact.json', 'r') as f:
    data = json.load(f)

# Extract relevant fields
formatted_data = []
data = data['train']  # Access the training split

In [36]:
from tqdm import tqdm
true, false = [], []
for i, item in tqdm(enumerate(data)):
    if item['answer'] == 1:
        true.append((item, i))
    else:
        false.append((item, i))

92283it [00:00, 3717684.04it/s]


In [42]:
import numpy as np
n = len(data)//8
true_array = np.array(true, dtype=object)
false_array = np.array(false, dtype=object)

# Sample x items from the true list
indices = np.random.choice(len(true_array), size=n, replace=False)
true_samples = true_array[indices]

# Sample x items from the false list
indices = np.random.choice(len(false_array), size=n, replace=False)
false_samples = false_array[indices]

In [52]:
all_data = np.concatenate((true_samples, false_samples), axis=0)
sorted_all_data = all_data[np.argsort(all_data[:, 1])]
formatted_data = []
indices = []
for item, i in sorted_all_data:
    indices.append(i)
    formatted_data.append({
        'prompt': f"{item['question']}\n{item['tables']['bracket']}",
        'completion': str(item['answer'])
    })
# Save as JSONL
jsonl_path = "formatted_dataset_balanced.jsonl"
with open(jsonl_path, "w", encoding="utf-8") as f:
    for entry in formatted_data:
        f.write(json.dumps(entry) + "\n")
        
with open("indices.json", "w") as file:
    json.dump(indices, file)
print(f"JSONL file saved as {jsonl_path}")

JSONL file saved as formatted_dataset_balanced.jsonl


In [45]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="./formatted_tabfact_balanced/formatted_dataset_balanced.jsonl", split="train")

In [12]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="formatted_dataset.jsonl", split="train")

Generating train split: 23070 examples [00:00, 526665.07 examples/s]


In [47]:
print(sum([1 for x in dataset['completion'] if x == '1']) / sum([1 for x in dataset['completion'] if x == '0']))
print(len(dataset))

1.0
23070


In [48]:
len(dataset['completion']) == len(dataset['prompt'])

True

In [53]:
def formatting_prompts_func(example):
    # Assert whether or not the question is True (1) or False (0) according to the table. 
    output_texts = []
    print(len(example['prompt'])) #WHY IS IT MISMATCHED IM TWEAKING
    print(len(example['completion']))
    for i in range(len(example['completion'])):
        text = f"Assert whether or not the question is True (1) or False (0) according to the table. ### Question: {example['prompt'][i]}\n ### Answer: {example['completion'][i]}"
        output_texts.append(text)
    # text = f"Assert whether or not the question is True (1) or False (0) according to the table. ### Question: {example['prompt']}\n ### Answer: {example['completion']}"
    # print(len(example['prompt']))
    return output_texts

In [38]:
def formatting_func(example):
    text = f"### Question: {example['prompt']}\n ### Answer: {example['completion']}"
    return text

In [55]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    formatting_func=formatting_prompts_func,
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 4,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "llama-3.2-3B-finetuned-tabfact_balanced",

        save_strategy="epoch",  # Save at the end of each epoch
        save_steps=-1,  # Automatically save at epoch end
        save_total_limit=-1,
    ),
)
trainer_stats = trainer.train()

[codecarbon ERROR @ 20:58:34] Error: Another instance of codecarbon is probably running as we find `/tmp/.codecarbon.lock`. Turn off the other instance to be able to run this one or use `allow_multiple_runs` or delete the file. Exiting.


553
1


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 23,070 | Num Epochs = 4 | Total steps = 2,884
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 24,313,856/1,865,526,272 (1.30% trained)
[codecarbon WARNING @ 20:58:35] Another instance of codecarbon is already running. Exiting.


Unsloth: Will smartly offload gradients to save VRAM!


[codecarbon INFO @ 20:58:44] Energy consumed for RAM : 0.001074 kWh. RAM Power : 23.44778537750244 W
[codecarbon INFO @ 20:58:44] Energy consumed for all CPUs : 0.001947 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:58:44] Energy consumed for all GPUs : 0.003378 kWh. Total GPU Power : 171.2241676980629 W
[codecarbon INFO @ 20:58:44] 0.006400 kWh of electricity used since the beginning.


Step,Training Loss
1,1.296600
2,1.304600
3,1.289100
4,1.327700
5,1.392400
6,1.436000
7,1.347800
8,1.354400
9,1.335600
10,1.319100


[codecarbon INFO @ 20:58:59] Energy consumed for RAM : 0.001172 kWh. RAM Power : 23.44778537750244 W
[codecarbon INFO @ 20:58:59] Energy consumed for all CPUs : 0.002125 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:58:59] Energy consumed for all GPUs : 0.004737 kWh. Total GPU Power : 326.1428796137384 W
[codecarbon INFO @ 20:58:59] 0.008034 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:59:14] Energy consumed for RAM : 0.001270 kWh. RAM Power : 23.44778537750244 W
[codecarbon INFO @ 20:59:14] Energy consumed for all CPUs : 0.002302 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:59:14] Energy consumed for all GPUs : 0.006140 kWh. Total GPU Power : 336.6946341189032 W
[codecarbon INFO @ 20:59:14] 0.009711 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:59:29] Energy consumed for RAM : 0.001367 kWh. RAM Power : 23.44778537750244 W
[codecarbon INFO @ 20:59:29] Energy consumed for all CPUs : 0.002479 kWh. Total CPU Power : 42.5 W
[codecarbo

: 

In [ ]:
from unsloth import FastLanguageModel
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint_path = "outputs/checkpoint-2500"
model = FastLanguageModel.from_pretrained(checkpoint_path)
tokenizer = model[1]
model = model[0]

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.50.0.dev0.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.64 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [16]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

checkpoint_path = "outputs/checkpoint-2500"

# Load model and tokenizer, ensuring they are on GPU
model, tokenizer = FastLanguageModel.from_pretrained(checkpoint_path)
model = FastLanguageModel.for_inference(model)
model.to("cuda")  # Move model to GPU


==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.50.0.dev0.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.64 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [23]:
# Sample input text
table = "[['Year', 'Division', 'League', 'Regular Season', 'Playoffs', 'Open Cup', 'Avg. Attendance'], ['2001', '2', 'USL A-League', '4th, Western', 'Quarterfinals', 'Did not qualify', '7,169'], ['2002', '2', 'USL A-League', '2nd, Pacific', '1st Round', 'Did not qualify', '6,260'], ['2003', '2', 'USL A-League', '3rd, Pacific', 'Did not qualify', 'Did not qualify', '5,871'], ['2004', '2', 'USL A-League', '1st, Western', 'Quarterfinals', '4th Round', '5,628'], ['2005', '2', 'USL First Division', '5th', 'Quarterfinals', '4th Round', '6,028'], ['2006', '2', 'USL First Division', '11th', 'Did not qualify', '3rd Round', '5,575'], ['2007', '2', 'USL First Division', '2nd', 'Semifinals', '2nd Round', '6,851'], ['2008', '2', 'USL First Division', '11th', 'Did not qualify', '1st Round', '8,567'], ['2009', '2', 'USL First Division', '1st', 'Semifinals', '3rd Round', '9,734'], ['2010', '2', 'USSF D-2 Pro League', '3rd, USL (3rd)', 'Quarterfinals', '3rd Round', '10,727']]"
question = "What was the last year where this team was a part of the USL A-League?"
answer = "2004"
input_text = f"Given this table: {table}. Answer the following. question: {question}, answer: {answer}\n question:"

# Encode the input text and move it to GPU
input_ids = tokenizer.encode(input_text, return_tensors="pt").to("cuda")

# Generate output using the model
output_ids = model.generate(input_ids, max_length=2048)

# Decode the output to text
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(f"Input: {input_text}")
print(f"Output: {output_text}")

Input: Given this table: [['Year', 'Division', 'League', 'Regular Season', 'Playoffs', 'Open Cup', 'Avg. Attendance'], ['2001', '2', 'USL A-League', '4th, Western', 'Quarterfinals', 'Did not qualify', '7,169'], ['2002', '2', 'USL A-League', '2nd, Pacific', '1st Round', 'Did not qualify', '6,260'], ['2003', '2', 'USL A-League', '3rd, Pacific', 'Did not qualify', 'Did not qualify', '5,871'], ['2004', '2', 'USL A-League', '1st, Western', 'Quarterfinals', '4th Round', '5,628'], ['2005', '2', 'USL First Division', '5th', 'Quarterfinals', '4th Round', '6,028'], ['2006', '2', 'USL First Division', '11th', 'Did not qualify', '3rd Round', '5,575'], ['2007', '2', 'USL First Division', '2nd', 'Semifinals', '2nd Round', '6,851'], ['2008', '2', 'USL First Division', '11th', 'Did not qualify', '1st Round', '8,567'], ['2009', '2', 'USL First Division', '1st', 'Semifinals', '3rd Round', '9,734'], ['2010', '2', 'USSF D-2 Pro League', '3rd, USL (3rd)', 'Quarterfinals', '3rd Round', '10,727']]. Answer th

In [28]:
# Sample input text
table = "[['Year', 'Division', 'League', 'Regular Season', 'Playoffs', 'Open Cup', 'Avg. Attendance'], ['2001', '2', 'USL A-League', '4th, Western', 'Quarterfinals', 'Did not qualify', '7,169'], ['2002', '2', 'USL A-League', '2nd, Pacific', '1st Round', 'Did not qualify', '6,260'], ['2003', '2', 'USL A-League', '3rd, Pacific', 'Did not qualify', 'Did not qualify', '5,871'], ['2004', '2', 'USL A-League', '1st, Western', 'Quarterfinals', '4th Round', '5,628'], ['2005', '2', 'USL First Division', '5th', 'Quarterfinals', '4th Round', '6,028'], ['2006', '2', 'USL First Division', '11th', 'Did not qualify', '3rd Round', '5,575'], ['2007', '2', 'USL First Division', '2nd', 'Semifinals', '2nd Round', '6,851'], ['2008', '2', 'USL First Division', '11th', 'Did not qualify', '1st Round', '8,567'], ['2009', '2', 'USL First Division', '1st', 'Semifinals', '3rd Round', '9,734'], ['2010', '2', 'USSF D-2 Pro League', '3rd, USL (3rd)', 'Quarterfinals', '3rd Round', '10,727']]"
question = "what was the name of the first team this team was a part of before becoming an independent team?"
answer = "2004"
input_text = f"Please read the following table and answer the questions. question: {question}, {table} answer:"

# Encode the input text and move it to GPU
input_ids = tokenizer.encode(input_text, return_tensors="pt").to("cuda")

# Generate output using the model
output_ids = model.generate(input_ids, max_length=2048)

# Decode the output to text
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(f"Input: {input_text}")
print(f"Output: {output_text}")

Input: Please read the following table and answer the questions. question: what was the name of the first team this team was a part of before becoming an independent team?, [['Year', 'Division', 'League', 'Regular Season', 'Playoffs', 'Open Cup', 'Avg. Attendance'], ['2001', '2', 'USL A-League', '4th, Western', 'Quarterfinals', 'Did not qualify', '7,169'], ['2002', '2', 'USL A-League', '2nd, Pacific', '1st Round', 'Did not qualify', '6,260'], ['2003', '2', 'USL A-League', '3rd, Pacific', 'Did not qualify', 'Did not qualify', '5,871'], ['2004', '2', 'USL A-League', '1st, Western', 'Quarterfinals', '4th Round', '5,628'], ['2005', '2', 'USL First Division', '5th', 'Quarterfinals', '4th Round', '6,028'], ['2006', '2', 'USL First Division', '11th', 'Did not qualify', '3rd Round', '5,575'], ['2007', '2', 'USL First Division', '2nd', 'Semifinals', '2nd Round', '6,851'], ['2008', '2', 'USL First Division', '11th', 'Did not qualify', '1st Round', '8,567'], ['2009', '2', 'USL First Division', '1